In [28]:
import glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import CARETLEFTBASE, CARETRIGHTBASE
from csv_dtypes import column_dtypes

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm, linear_model
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures

pd.set_option("display.max_rows", 60)  # default 60
pd.set_option("display.max_columns", 20)  # default 20

In [29]:
all_files = glob.glob("../roll_data/*-steal_attempt.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

seasons = [12]
# seasons = [11, 12, 13, 14]
# seasons = [13]
# seasons = [14]
df = df[df["season"].isin(seasons)]
len(df)

102747

In [31]:
def get_pitcher_mul(row):
    pm = str(row["pitcher_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    if "OVERPERFORMING" in pm:
        mul += 0.2
    if "OVERPERFORMING" in ptm:
        mul += 0.2
    if "UNDERPERFORMING" in pm:
        mul -= 0.2
    if "UNDERPERFORMING" in ptm:
        mul -= 0.2
    if "TRAVELING" in ptm:
        # pass  # traveling just doesn't do anything on pitchers?
        if not row["top_of_inning"]:
            mul += 0.05
    if "HIGH_PRESSURE" in ptm:
        if row["weather"] == "Weather.FLOODING" and row["baserunner_count"] > 0:
            mul += 0.25
    if "GROWTH" in ptm:
        mul += min(0.05, 0.05 * (row["day"] / 99))
        # pass  # growth doesn't do anything on pitchers either
    if "SINKING_SHIP" in ptm:
        mul += (14 - row["pitching_team_roster_size"]) * 0.01
    if "AFFINITY_FOR_CROWS" in ptm and row["weather"] == "Weather.BIRDS":
        mul += 0.5
    return mul


def get_batter_mul(row):
    # if row[["batter_name"]].isna().any():
    # row["batter_name"] == "NaaN"
    bm = str(row["batter_mods"])
    btm = str(row["batting_team_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    attrs = ""
    if "OVERPERFORMING" in bm:
        mul += 0.2
    if "OVERPERFORMING" in btm:
        mul += 0.2
    if "UNDERPERFORMING" in bm:
        mul -= 0.2
    if "UNDERPERFORMING" in btm:
        mul -= 0.2
    if "TRAVELING" in btm:
        if row["top_of_inning"]:
            # is this actually 0.035 instead of 0.05?
            mul += 0.35
    if "GROWTH" in btm:
        # is this actually 0.035 instead of 0.05?
        mul += min(0.05, 0.05 * (row["day"] / 99))
        # pass
    if "HIGH_PRESSURE" in btm:
        if row["weather"] == "Weather.FLOODING" and row["baserunner_count"] > 0:
            mul += 0.25
    if "SINKING_SHIP" in btm:
        mul += (14 - row["batting_team_roster_size"]) * 0.01
    if "AFFINITY_FOR_CROWS" in btm and row["weather"] == "Weather.BIRDS":
        mul += 0.5
    if "CHUNKY" in bm and row["weather"] == "Weather.PEANUTS":
        # todo: make this work if there's also, like, Overperforming active
        # it only should apply to "power" attrs
        mul += 1.0
        attrs = "batter_musclitude, batter_divinity, batter_ground_friction"
    if "SMOOTH" in bm and row["weather"] == "Weather.PEANUTS":
        # todo: make this work if there's also, like, Overperforming active
        # it only should apply to "speed" attrs
        # doing 80% because of evidence from the Spin Attack blessing that "speed"
        # boosts laserlikeness 80% of what it says the total boost is?
        mul += 0.80
        attrs = "batter_musclitude, batter_laserlikeness, batter_ground_friction, batter_continuation"
    if "ON_FIRE" in bm:
        # todo: handle
        # test "+125% multiplier"
        mul += 1.25
        # pass
    # NVGs real??? eclipse weather, special case for Sutton Dreamy
    # surely this will not break from scattering, right? (It totally will)
    # if row["batter_name"] == "Sutton Dreamy" and row["weather"] == "Weather.ECLIPSE":
    # NVGs allow the player to play 50% better in a solar eclipse.
    # This might affect fielding and baserunning as well?
    # mul += 0.50
    return mul, attrs


def get_fielder_mul(row):
    fm = str(row["fielder_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    if "OVERPERFORMING" in fm:
        mul += 0.2
    if "OVERPERFORMING" in ptm:
        mul += 0.2
    if "UNDERPERFORMING" in fm:
        mul -= 0.2
    if "UNDERPERFORMING" in ptm:
        mul -= 0.2
    if "TRAVELING" in ptm:
        # pass
        if not row["top_of_inning"]:
            mul += 0.05
    if "HIGH_PRESSURE" in ptm:
        if row["weather"] == 18 and str(row["baserunner_count"]) > 0:
            mul += 0.25
    if "GROWTH" in ptm:
        mul += min(0.05, 0.05 * (row["day"] / 99))
    if "SINKING_SHIP" in ptm:
        mul += (14 - row["pitching_team_roster_size"]) * 0.01
    if "AFFINITY_FOR_CROWS" in ptm and row["weather"] == "Weather.BIRDS":
        mul += 0.5
    if "SHELLED" in fm:
        # is it this, or is it "mul = 0", I wonder
        mul -= 1.0
    # if row["fielder_name"] == "Sutton Dreamy" and row["weather"] == "Weather.ECLIPSE":
    # NVGs allow the player to play 50% better in a solar eclipse.
    # This might affect fielding and baserunning as well?
    # mul += 0.50
    return mul

In [33]:
# df["batter_mul"] = df.apply(get_batter_mul, axis=1)
df[["batter_mul", "batter_mul_attrs"]] = df.apply(get_batter_mul, axis=1, result_type="expand")
df["pitcher_mul"] = df.apply(get_pitcher_mul, axis=1)
df["fielder_mul"] = df.apply(get_fielder_mul, axis=1)

In [34]:
df.loc[df["pitcher_mods"].astype(str).str.contains("SCATTERED"), "pitcher_vibes"] = 0
df.loc[df["batter_mods"].astype(str).str.contains("SCATTERED"), "batter_vibes"] = 0
df.loc[df["fielder_mods"].astype(str).str.contains("SCATTERED"), "fielder_vibes"] = 0

df["incon_center"] = df["ballpark_inconvenience"] - 0.5
df["elong_center"] = df["ballpark_elongation"] - 0.5

In [35]:
for attr in [
    "batter_base_thirst",
    "batter_continuation",
    "batter_ground_friction",
    "batter_indulgence",
    "batter_laserlikeness",
]:
    # had to do all this to make chunky and smooth work correctly
    df[attr + "_mul_vibe"] = df.apply(
        lambda x: x[attr] * x["batter_mul"] * (1 + 0.2 * x["batter_vibes"])
        if (x["batter_mul_attrs"] == "") or (attr in x["batter_mul_attrs"])
        else x[attr] * (1 + 0.2 * x["batter_vibes"]),
        axis=1,
    )
for attr in [
    # "batter_patheticism",
    # "batter_tragicness",
]:
    # had to do all this to make chunky and smooth work correctly
    df[attr + "_mul_vibe"] = df.apply(
        lambda x: x[attr] / x["batter_mul"] * (1 + 0.2 * x["batter_vibes"])
        if (x["batter_mul_attrs"] == "") or (attr in x["batter_mul_attrs"])
        else x[attr] * (1 + 0.2 * x["batter_vibes"]),
        axis=1,
    )
    # df[attr + "_mul_vibe"] = df[attr] * df["batter_mul"] * (1 + 0.2 * df["batter_vibes"])

In [36]:
df = df.copy()
for attr in [
    "pitcher_anticapitalism",
    "pitcher_chasiness",
    "pitcher_omniscience",
    "pitcher_tenaciousness",
    "pitcher_watchfulness",
]:
    # df[attr+"_mul_vibe"] = df.apply(lambda x: x[attr] * x["pitcher_mul"] * (1 + 0.2 * df["pitcher_vibes"])
    #                           if (x["pitcher_mul_attrs"] == "") or (attr in x["pitcher_mul_attrs"])
    #                           else x[attr] * (1 + 0.2 * df["pitcher_vibes"]),
    #                           axis=1)
    df[attr + "_mul_vibe"] = df[attr] * df["pitcher_mul"] * (1 + 0.2 * df["pitcher_vibes"])

In [37]:
if "fielder_vibes" in df:
    for attr in [
        "fielder_anticapitalism",
        "fielder_chasiness",
        "fielder_omniscience",
        "fielder_tenaciousness",
        "fielder_watchfulness",
    ]:
        # df[attr+"_mul_vibe"] = df.apply(lambda x: x[attr] * x["fielder_mul"] * (1 + 0.2 * df["fielder_vibes"])
        #                       if (x["fielder_mul_attrs"] == "") or (attr in x["fielder_mul_attrs"])
        #                       else x[attr] * (1 + 0.2 * df["fielder_vibes"]),
        #                       axis=1)
        df[attr + "_mul_vibe"] = df[attr] * df["fielder_mul"] * (1 + 0.2 * df["fielder_vibes"])

df = df.copy()

In [38]:
df.groupby("event_type").size().sort_values(ascending=False)[0:15]

event_type
StealAttempt0    51496
StealAttempt1    26262
StealAttempt2    24989
dtype: int64

In [39]:
dfc = df[df["event_type"] == "StealAttempt0"].copy()
for exclude_mod in [
    "OVERPERFORMING",
    "UNDERPERFORMING",
    "HIGH_PRESSURE",
    "GROWTH",
    "SINKING_SHIP",
    "TRAVELING",
    "ON_FIRE",
    "CHUNKY",
    "SMOOTH",
]:
    # for exclude_mod in ["FLINCH", "O_NO", "ON_FIRE", "GROWTH", "TRAVELING"]:
    # for exclude_mod in ["SPICY", "ON_FIRE"]:
    dfc = dfc[~dfc["batter_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitcher_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["fielder_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitching_team_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["batting_team_mods"].astype(str).str.contains(exclude_mod)]

# dfc["laser_thirst"] = dfc["batter_laserlikeness_mul_vibe"] * dfc["batter_base_thirst_mul_vibe"]
# dfc["laser_thirst"] = dfc["batter_laserlikeness_mul_vibe"] * dfc["batter_base_thirst"]
# dfc["laser_thirst"] = dfc["batter_laserlikeness"] * dfc["batter_base_thirst"]


dfc = dfc[(dfc["pitcher_mul"] == 1) & (dfc["batter_mul"] == 1) & (dfc["fielder_mul"] == 1)].copy()

# dfc = dfc[dfc["stadium_id"].isna()].copy()

dfc = dfc[dfc["roll"] > 0.001].copy()

# dfc["roll_log"] = np.log(dfc["roll"])
# dfc["lt_log"] = np.log(dfc["laser_thirst"])
# dfc["laser_log"] = np.log(dfc["batter_laserlikeness"])

# dfc["thirst_log"] = np.log(dfc["batter_base_thirst"])

# dfc["watch"] = (
#     # 3 * dfc["pitcher_watchfulness_mul_vibe"]
#     # + 1 * dfc["fielder_watchfulness_mul_vibe"]
#     3 * dfc["pitcher_watchfulness"]
#     + 1 * dfc["fielder_watchfulness"]
# ) / 1
# dfc["watch_log"] = np.log(dfc["watch"])
# dfc["watch_pow"] = dfc["watch"]**1.5
# dfc["lt_pow"] = dfc["laser_thirst"]**1.5

# dfc = dfc[dfc["laser_thirst"] > 1e-3].copy()
len(dfc)
# dfc["laser_thirst"].min()
# dfc.groupby("ballpark_inconvenience").size()

36579

In [40]:
dfc["pwatch"] = dfc["pitcher_watchfulness"]  # * (1 + 0.2*dfc["pitcher_vibes"])
dfc["fwatch"] = dfc["fielder_watchfulness"] * (1 + 0.2 * dfc["fielder_vibes"])
dfc["watch"] = (3 * dfc["pwatch"] + dfc["fwatch"]) / 4

# dfc["roll_mod"] = (dfc["roll"] - 0.05 + 0.08 * dfc["watch"])
dfc["roll_mod"] = dfc["roll"] + 0.08 * dfc["watch"]

dfc["laser_vibe"] = dfc["batter_laserlikeness_mul_vibe"]
dfc["laser"] = dfc["batter_laserlikeness_mul_vibe"]
dfc["lasersq"] = dfc["batter_laserlikeness_mul_vibe"] ** 2
dfc["lasersq_thirst"] = dfc["lasersq"] * dfc["batter_base_thirst"]

dfc["laser_thirst"] = dfc["batter_laserlikeness_mul_vibe"] * dfc["batter_base_thirst"]
dfc["laser_thirstsq"] = dfc["batter_laserlikeness_mul_vibe"] * (dfc["batter_base_thirst"] ** 2)
dfc["thirstsq"] = dfc["batter_base_thirst"] ** 2
dfc["thirst"] = dfc["batter_base_thirst"]
dfc["thirstpow"] = dfc["batter_base_thirst"] ** 0.9


dfc["park_factors"] = dfc["incon_center"] + 2 * dfc["elong_center"]
# dfc["park_factors"] = dfc["ballpark_inconvenience"] + 2 * dfc["ballpark_elongation"]

# dfc["laser_thirst_term"] = dfc["thirst"] * (dfc["laser"]**1.7)
dfc["laser_thirst_term"] = dfc["thirst"] * (3 * dfc["laser"] ** 2 + dfc["laser"]) / 4

dfc["thirst_elong"] = dfc["thirst"] * dfc["ballpark_elongation"]
dfc["thirst_incon"] = dfc["thirst"] * dfc["ballpark_inconvenience"]
dfc["laser_elong"] = dfc["laser"] * dfc["ballpark_elongation"]
dfc["laser_incon"] = dfc["laser"] * dfc["ballpark_inconvenience"]

# dfc["roll_log"] = np.log(dfc["roll_mod"])
# dfc["laser_log"] = np.log(0.205 + dfc["laser"]* (1+0.2*dfc["runner_vibes"]))


In [41]:
X = dfc[
    [
        "passed",
        # "roll",
        "roll_mod",
        # "laser",
        "lasersq",
        # "laser_thirst",
        # "lasersq_thirst",
        "laser_thirst_term",
        # "laser_thirstsq",
        # "thirst",
        # "thirstsq",
        "thirstpow",
        # "laser_elong",
        # "laser_incon",
        # "watch",
        # "pwatch",
        # "fwatch",
        # "watch_log",
        # "watch_pow",
        "park_factors",
        # "incon_center",
        # "elong_center",
    ]
]
# X = X[X['roll'] < 0.85]
# X = X[~X["roll_adj_log"].isna()]
y = X["passed"]
y = y.astype("int")
X = X.drop("passed", axis=1)

pin_intercept = None
# pin_intercept = 0.042838197858485604
pins = [
]
for val, var in pins:
    X["roll_mod"] -= X[var] * val
    X = X.drop(var, axis=1)

sc = StandardScaler(with_mean=False)
X2 = sc.fit_transform(X)
if pin_intercept is not None:
    X2[:, 0] -= pin_intercept / sc.scale_[0]

trainedsvm = svm.LinearSVC(
    dual=False, max_iter=10000000, C=100000000, tol=1e-12, fit_intercept=pin_intercept is None
).fit(X2, y)
predictionsvm = trainedsvm.predict(X2)
print(confusion_matrix(y, predictionsvm))
print(confusion_matrix(y, predictionsvm)[0, 1] + confusion_matrix(y, predictionsvm)[1, 0], "outliers")
print(trainedsvm.score(X2, y))

coef = np.true_divide(trainedsvm.coef_, sc.scale_)
coef_scaled = coef / coef[0, 0]
coef_list = coef_scaled.tolist()[0]

intercept = trainedsvm.intercept_  # - np.dot(coef, sc.mean_)
intercept_scaled = -(intercept / coef[0, 0])[0] if pin_intercept is None else pin_intercept

dfc["threshold"] = intercept_scaled
print("intercept:", intercept_scaled)
for pair in pins + list(zip(-np.array(coef_list), X.columns)):
    if "roll" not in pair[1]:
        print(pair)
        dfc["threshold"] += pair[0] * dfc[pair[1]]

[[35033     4]
 [    4  1538]]
8 outliers
0.9997812952787118
intercept: 0.04301953506404355
(0.010851314764287738, 'lasersq')
(0.1006233259026676, 'laser_thirst_term')
(0.017224762013959536, 'thirstpow')
(-0.03200393961381601, 'park_factors')


In [59]:
%matplotlib notebook
import ipywidgets

y_val = "roll_mod"
x_val = "laser"
filter_attr = "thirst"
fig, ax = plt.subplots(1, figsize=(6, 6))
# in theory i should be able to plot empty arrays and have update() fill them but that
# doesn't work for some reason
(no_steal_plot,) = ax.plot(
    dfc[(~dfc["passed"]) & (abs(dfc[filter_attr] - 0.5) < 0.1)][x_val],
    dfc[(~dfc["passed"]) & (abs(dfc[filter_attr] - 0.5) < 0.1)][y_val],
    color="red",
    label="No Steal",
    marker=CARETRIGHTBASE,
    ls="",
)
(yes_steal_plot,) = ax.plot(
    dfc[(dfc["passed"]) & (abs(dfc[filter_attr] - 0.5) < 0.1)][x_val],
    dfc[(dfc["passed"]) & (abs(dfc[filter_attr] - 0.5) < 0.1)][y_val],
    color="blue",
    label="Steal",
    marker=CARETLEFTBASE,
    ls="",
)
ax.set_xlabel(x_val)
ax.set_ylabel(y_val)
ax.set_ylim(0, 0.2)


def update(filter_val=0.5, window_size=0.1):
    no_steal_plot.set_xdata(dfc[(~dfc["passed"]) & (abs(dfc[filter_attr] - filter_val) < window_size)][x_val])
    no_steal_plot.set_ydata(dfc[(~dfc["passed"]) & (abs(dfc[filter_attr] - filter_val) < window_size)][y_val])
    yes_steal_plot.set_xdata(dfc[(dfc["passed"]) & (abs(dfc[filter_attr] - filter_val) < window_size)][x_val])
    yes_steal_plot.set_ydata(dfc[(dfc["passed"]) & (abs(dfc[filter_attr] - filter_val) < window_size)][y_val])
    ax.set_title(f"Steal Attempts with {filter_val - window_size:.3f} < {filter_attr} < {filter_val + window_size:.3f}")
    fig.canvas.draw_idle()


update()

max_filter_val = dfc[filter_attr].max()
ipywidgets.interact(update, filter_val=(0.0, max_filter_val, 0.01), window_size=(0.0, 0.5, 0.01));
# Uncomment this to save a gif of it
# total_steps = 60 * 5
# for i in range(total_steps):
#     filter_val = i / (total_steps - 1) * max_filter_val
#     update(filter_val, window_size=0.05)
#     fig.savefig(f"output/steal-attempt-constant-{filter_attr}-{i:03d}.png")

<IPython.core.display.Javascript object>

interactive(children=(FloatSlider(value=0.5, description='filter_val', max=1.7730689766234986, step=0.01), Flo…

In [ ]:
# season 13, null stadium only
# [[26065     0]
#  [    0  1153]]
# 0 outliers
# 1.0
# intercept: 0.043272646512567126
# (0.002301879255231324, 'laser_vibe'),
# (0.0076657470915077715, 'lasersq'),
# (0.0269960061526303, 'laser_thirst'),
# (0.07758523709024914, 'lasersq_thirst'),
# (-0.0032036086834798284, 'laser_thirstsq'),
# (0.0171756417623993, 'batter_base_thirst'),
# (-0.06039525672661708, 'pwatch'),
# (-0.019936549618782263, 'fwatch'),

# dfc["threshold"] = intercept_scaled[0]
# print("intercept:", intercept_scaled)
# for pair in pins + list(zip(-np.array(coef_list), X.columns)):
#     if "roll" not in pair[1]:
#         print(pair)
#         dfc["threshold"] += pair[0] * dfc[pair[1]]

y_val = "roll_mod"
x_val = "threshold"
# x_val = "watch"
fig, ax = plt.subplots(1, figsize=(6, 6))
ax.scatter(
    dfc[~dfc["passed"]][x_val],
    dfc[~dfc["passed"]][y_val],
    color="red",
    label="No Steal",
    marker=CARETRIGHTBASE,
)
ax.scatter(
    dfc[dfc["passed"]][x_val],
    dfc[dfc["passed"]][y_val],
    color="blue",
    label="Steal",
    marker=CARETLEFTBASE,
)
ax.set_xlabel(x_val)
ax.set_ylabel("roll")
ax.set_title(f"Season N Steal Attempts")
# ax.set_xlim(0.484, 0.499)
# ax.set_ylim(-0.01, 0.01)
# ax.set_xlim(-0.1, 0.1)
# ax.set_yscale("log")
# ax.set_xscale("log")
# if x_val == "threshold":
ax.plot(ax.get_xlim(), ax.get_xlim())

ax.legend()

In [ ]:
dfc["offset"] = dfc["roll_mod"] - dfc["threshold"]
outliers = dfc[(dfc["passed"] & (dfc["offset"] >= 0)) | (~dfc["passed"] & (dfc["offset"] <= 0))]

fig, ax = plt.subplots(1)
x_val = "threshold"
x_val = "thirst"
# x_val = "elong_center"
x_val = "park_factors"
ax.scatter(
    outliers[~outliers["passed"]][x_val],
    outliers[~outliers["passed"]]["offset"],
    color="red",
    label="No Steal",
)
ax.scatter(
    outliers[outliers["passed"]][x_val],
    outliers[outliers["passed"]]["offset"],
    color="blue",
    label="Steal",
)
ax.set_xlabel(x_val)
ax.set_ylabel("offset")
# ax.legend()
# ax.set_ylim(0,2)

In [ ]:
# pd.set_option("display.max_rows", None)  # default 60
pd.set_option("display.max_columns", 20)  # default 20
table = outliers[
    [
        "passed",
        "offset",
        "roll",
        "threshold",
        # "event_type",  # "home_score", "away_score", "top_of_inning", "inning",
        # "pitcher_mul",
        # "batter_mul",
        # "fielder_mul",  # "baserunner_count",
        # "batter_vibes", "pitcher_vibes",
        "laser",
        "thirst",
        "laser_thirst_term",
        "watch",
        "incon_center",
        "elong_center",
        "park_factors",
        "batter_name",
        "pitcher_name",
        "fielder_name",
        # "batter_mods",
        # "pitcher_mods",
        # "fielder_mods",
        # "batting_team_mods",
        # "pitching_team_mods",
        "season",
        "day",
        "game_id",
        "play_count",
        "stadium_id",
        # "fielder_roll",
    ]
]
print(len(table))
print(table.groupby("season").size())
table.sort_values("offset", ascending=False)  # [0:10]

In [ ]:
plt.rcParams["font.size"] = 10
x_val = "threshold"
fig, axes = plt.subplots(3, 2, figsize=(10, 15), constrained_layout=True)
# fig.suptitle(f"Season {season+1} Hits vs. Outs", fontsize=16)
dimlist = [5e-2, 2e-2, 1e-2, 0.5e-2, 0.2e-2, 0.1e-2]
for i, ax in enumerate(np.ravel(axes)):
    ax.scatter(
        dfc[dfc["passed"]][x_val],
        dfc[dfc["passed"]]["offset"],
        color="blue",
        label="Steal",
    )
    ax.scatter(
        dfc[~dfc["passed"]][x_val],
        dfc[~dfc["passed"]]["offset"],
        color="red",
        label="No Steal",
    )
    ax.scatter(
        outliers[~outliers["passed"]][x_val],
        outliers[~outliers["passed"]]["offset"],
        color="red",
        edgecolor="black",
    )
    ax.scatter(
        outliers[outliers["passed"]][x_val],
        outliers[outliers["passed"]]["offset"],
        color="blue",
        edgecolor="black",
    )
    ax.set_xlabel(x_val)
    ax.set_ylabel("offset")
    ax.legend()
    ax.grid()
    ax.set_ylim(-dimlist[i], dimlist[i])
# fig.savefig("../figures/hit_out_fit_offsets.png", facecolor='white')